In [1]:
import os

list_file = sorted(list(os.walk('./file/3.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/3.vehicle/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：507


['30250S',
 '30250SX',
 '30420S',
 '30430S',
 '30460S',
 '30460SA',
 '30460SB',
 '30470S',
 '30470SX',
 '30490S',
 '30490SA',
 '30490SAX',
 '30490SX',
 '30719S',
 '30719SHWP',
 '31007S',
 '31035S',
 '31035SX',
 '31036S',
 '31036SA',
 '31038S',
 '31038SX',
 '31041S',
 '31045S',
 '31045SA',
 '31045SX',
 '31047S',
 '31047SH',
 '31047SHWP',
 '31047SK3',
 '31047SVVT',
 '31047SX',
 '31047SXH',
 '31047SXHWP',
 '31049S',
 '31049SA',
 '31049SAH',
 '31049SAVVT1',
 '31049SAW',
 '31049SAWVVT',
 '31049SAWVVT1',
 '31049SB',
 '31049SBH',
 '31049SBX',
 '31049SBXH',
 '31049SC',
 '31049SCH',
 '31049SCX',
 '31049SCXH',
 '31049SD',
 '31049SDH',
 '31049SDX',
 '31049SDXH',
 '31049SE',
 '31049SEH',
 '31049SEK3',
 '31049SEVVT',
 '31049SF',
 '31049SFH',
 '31049SFVVT',
 '31049SG',
 '31049SGX',
 '31049SH',
 '31049SJ',
 '31049SK',
 '31049SL',
 '31049SM',
 '31049SN',
 '31049SP',
 '31049SPVVT',
 '31049SVVT1',
 '31049SVVT2',
 '31049SW',
 '31051S',
 '31051SA',
 '31051SAX',
 '31051SX',
 '31054S',
 '31055S',
 '31055SX'

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(input_.loc[a, 'Info'],
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))
        
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            if html.xpath('//span[@class="la-btn-v"]/text()')[0] == 'Continue Shopping':
                with open('./info.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1
2


<Response [200]>

In [8]:
import re

dict_specification = {}
list_tr = html.xpath('//table[@class="moreinfotable"]/tr')[1:] if html.xpath('//table[@class="moreinfotable"]/tr') else []
for i, tr in zip(range(len(list_tr)), list_tr):
    dict_specification[str(i)] = {}

    list_td = [re.sub(r'<.*?>', '', etree.tostring(text).decode('utf-8')).strip() for text in tr.xpath('./td')]
    for j, td in zip(range(len(list_td)), list_td):
        dict_specification[str(i)][str(j)] = td

dict_specification

{'0': {'0': 'Camshaft Sprocket Material', '1': 'High Alloy Steel'},
 '1': {'0': 'Keyway Adjustable', '1': 'No'},
 '2': {'0': 'Sprockets Included', '1': 'Yes'},
 '3': {'0': 'Timing Chain Type', '1': 'Silent'}}

In [9]:
import json

dict_kit = {}
list_tr = html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or (contains(@class, "altrow-b-1")))]/tr')
for i, tr in zip(range(len(list_tr)), list_tr):
    dict_kit[str(i)] = {}

    list_td = tr.xpath('./td')
    dict_kit[str(i)]['quantity'] = int(list_td[0].xpath('./text()')[0].strip())

    dict_kit[str(i)]['tag'] = list_td[1].xpath('./text()')[0].strip()

    json_ = json.loads(list_td[2].xpath('./@value')[0])
    dict_kit[str(i)]['type_code'] = json_['parttype']
    dict_kit[str(i)]['key'] = json_['partkey']

    json_2 = json.loads(tr.xpath('./input[contains(@id, "listing_data_supplemental[")]/@value')[0])
    dict_kit[str(i)]['part_number'] = json_2['partnumber']
    dict_kit[str(i)]['manufacturer'] = json_2['catalogname']

    dict_kit[str(i)]['url'] = f'''https://www.rockauto.com/en/parts/{json_2['catalogname']},{json_2['partnumber']},{list_td[1].xpath('./text()')[0].strip()},{json_['parttype']}'''

    list_info = tr.xpath('./div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]/@href')
    dict_kit[str(i)]['info'] = list_info[0].strip() if list_info and list_info[0].startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={json_['partkey']}&pt={json_['parttype']}&Lennon=1'''

    dict_src = {}
    list_src = json.loads(list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]/@value')[0])['Slots'] if list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]/@value') else []
    for j, src in zip(range(len(list_src)), list_src):
        dict_src[str(j)] = 'https://www.rockauto.com' + src['ImageData']['Full'].strip()
    dict_kit[str(i)]['dict_src'] = dict_src

dict_kit

{'0': {'quantity': 1,
  'tag': 'Timing Chain',
  'type_code': '5724',
  'key': '8306448',
  'part_number': '715F',
  'manufacturer': 'MELLING',
  'url': 'https://www.rockauto.com/en/parts/MELLING,715F,Timing Chain,5724',
  'info': 'https://www.rockauto.com/en/moreinfo.php?pk=8306448&cc=0&pt=5724',
  'dict_src': {'0': 'https://www.rockauto.com/info/583/715F-1-ANG.jpg'}},
 '1': {'quantity': 1,
  'tag': 'Timing Chain Guide / Damper',
  'type_code': '5728',
  'key': '8307424',
  'part_number': 'BG5515',
  'manufacturer': 'MELLING',
  'url': 'https://www.rockauto.com/en/parts/MELLING,BG5515,Timing Chain Guide / Damper,5728',
  'info': 'https://www.rockauto.com/en/moreinfo.php?pk=8307424&cc=0&pt=5728',
  'dict_src': {'0': 'https://www.rockauto.com/info/583/BG5515-1-SID.jpg'}},
 '2': {'quantity': 1,
  'tag': 'Timing Chain Tensioner',
  'type_code': '5736',
  'key': '8308132',
  'part_number': 'BT5514',
  'manufacturer': 'MELLING',
  'url': 'https://www.rockauto.com/en/parts/MELLING,BT5514,Tim

In [10]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                         'Group': input_.loc[a, 'Group'],
                         'Type': input_.loc[a, 'Type'],
                         'Type Code': input_.loc[a, 'Type Code'],
                         'Part Number': input_.loc[a, 'Part Number'],
                         'Parameter': input_.loc[a, 'Parameter'],
                         'Json_Price': input_.loc[a, 'Json_Price'],
                         'Note 1': input_.loc[a, 'Note 1'],
                         'Note 2': input_.loc[a, 'Note 2'],
                         'Json_Vehicle': input_.loc[a, 'Json_Vehicle'],
                         'OEM': input_.loc[a, 'OEM'],
                         'Picture': input_.loc[a, 'Picture'],
                         'Url': input_.loc[a, 'Url'],
                         'Info': input_.loc[a, 'Info'],
                         'Json_Src': input_.loc[a, 'Json_Src'],
                         'Json_Specification': json.dumps(dict_specification),
                         'Json_Kit': json.dumps(dict_kit),
                         'Key': input_.loc[a, 'Key']}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Manufacturer,Group,Type,Type Code,Part Number,Parameter,Json_Price,Note 1,Note 2,Json_Vehicle,OEM,Picture,Url,Info,Json_Src,Json_Specification,Json_Kit,Key
0,1,MELLING,Engine,Timing Chain & Component Kit,5756,30250S,,"{""0"": ""$172.79""}",,Category: Timing Chain &amp; Component Kit,"{""0"": {""0"": ""MAZDA"", ""1"": ""3"", ""2"": ""2004-2007...",,,"https://www.rockauto.com/en/parts/melling,3025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,"{""0"": ""https://www.rockauto.com/info/583/3-025...","{""0"": {""0"": ""Camshaft Sprocket Material"", ""1"":...","{""0"": {""quantity"": 1, ""tag"": ""Timing Chain"", ""...",8304960


In [11]:
crawler_status = 'ok'

crawler_status

'ok'

In [12]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [13]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_info.xlsx', index=False)

output_correct

,No,Manufacturer,Group,Type,Type Code,Part Number,Parameter,Json_Price,Note 1,Note 2,Json_Vehicle,OEM,Picture,Url,Info,Json_Src,Json_Specification,Json_Kit,Key
0,1,MELLING,Engine,Timing Chain & Component Kit,5756,30250S,,"{""0"": ""$172.79""}",,Category: Timing Chain &amp; Component Kit,"{""0"": {""0"": ""MAZDA"", ""1"": ""3"", ""2"": ""2004-2007...",,,"https://www.rockauto.com/en/parts/melling,3025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,"{""0"": ""https://www.rockauto.com/info/583/3-025...","{""0"": {""0"": ""Camshaft Sprocket Material"", ""1"":...","{""0"": {""quantity"": 1, ""tag"": ""Timing Chain"", ""...",8304960


In [14]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_info_error.xlsx', index=False)

output_error

""
